In [94]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

import json

np.__version__

'1.19.5'

# Compress data to unique songs, add a "contains 'love'" column for reference

In [95]:
gender_df = pd.read_csv('./data/11-OUTPUT-with-gender-from-gpt4.csv')
date_as_decimal = (gender_df.chart_debut_date_normalized_0_to_1 / 100) * (2023 - 1959) + 1959
date_as_decimal
gender_df['date_as_decimal'] = date_as_decimal
gender_df

,performer,song,generic_genre,lyric_line,chart_debut,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,near_neighbors_count,gender,date_as_decimal
0,John Lennon,#9 Dream,ROCK,So long ago,1974-12-21,62.357605,41.475302,25.450,10.0,14,m,1975.28800
1,John Lennon,#9 Dream,ROCK,"Was it in a dream, was it just a dream?",1974-12-21,55.684884,27.032071,25.450,10.0,14,m,1975.28800
2,John Lennon,#9 Dream,ROCK,"I know, yes I know",1974-12-21,69.964586,46.613546,25.450,10.0,0,m,1975.28800
3,John Lennon,#9 Dream,ROCK,"Seemed so very real, it seemed so real to me",1974-12-21,60.181471,69.527274,25.450,10.0,14,m,1975.28800
4,John Lennon,#9 Dream,ROCK,Took a walk down the street,1974-12-21,49.938720,41.965371,25.450,10.0,0,m,1975.28800
...,...,...,...,...,...,...,...,...,...,...,...,...
167355,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"I'm addicted to promethazine, it's crazy, yeah...",2021-05-29,41.610927,50.449968,97.562,20.0,0,m,2021.43968
167356,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"All this money coming in, it drive me crazy no...",2021-05-29,45.775471,52.982471,97.562,20.0,0,m,2021.43968
167357,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,I'll be crazy if I blow it,2021-05-29,45.995316,47.344963,97.562,20.0,5,m,2021.43968
167358,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,And it left so many R.I.P.,2021-05-29,57.715991,46.589529,97.562,20.0,0,m,2021.43968


In [96]:
IS_LOVE_SONG_COL = 'is_love_song'

def contains_love(lyric_lines):
    # Short-circuit as soon as "love" is found in any lyric line.
    return any('love' in lyric.lower() for lyric in lyric_lines)

def list_unique_songs_labeled_if_they_contain_the_word_love(df):
        # Create a boolean mask indicating whether each lyric line contains "love".
    df['contains_love'] = df['lyric_line'].str.contains('love', case=False)
    
    # Define aggregation functions for each column
    # - 'contains_love' will use 'any' to find if any lyric line contains 'love'
    # - other columns will use 'first' to take the first value from each group
    agg_functions = {col: 'first' for col in df.columns if col not in ['contains_love', 'lyric_line']}
    agg_functions['contains_love'] = 'any'
    
    # Group by 'performer' and 'song' and apply aggregation functions
    result = df.groupby(['performer', 'song']).agg(agg_functions)#.reset_index()
    
    # Rename 'contains_love' column to 'is_love_song'
    result.rename(columns={'contains_love': IS_LOVE_SONG_COL}, inplace=True)
    
    return result.reset_index(drop=True)

# Example DataFrame with additional columns
data = {
    'lyric_line': ['I love you', 'This is a test', 'Love is in the air', 'No mentions here', 'Love', 'Something else'],
    'performer': ['Artist1', 'Artist1', 'Artist2', 'Artist2', 'Artist3', 'Artist3'],
    'song': ['Song1', 'Song1', 'Song2', 'Song2', 'Song3', 'Song3'],
    'album': ['Album1', 'Album1', 'Album2', 'Album2', 'Album3', 'Album3'],
    'year': [2000, 2000, 2005, 2005, 2010, 2010]
}

# Create DataFrame
df = pd.DataFrame(data)

# Transform DataFrame
list_unique_songs_labeled_if_they_contain_the_word_love(df)

,performer,song,album,year,is_love_song
0,Artist1,Song1,Album1,2000,True
1,Artist2,Song2,Album2,2005,True
2,Artist3,Song3,Album3,2010,True


In [97]:
contains_love_df = list_unique_songs_labeled_if_they_contain_the_word_love(gender_df)
contains_love_df

,performer,song,generic_genre,chart_debut,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,near_neighbors_count,gender,date_as_decimal,is_love_song
0,"""Weird Al"" Yankovic",White & Nerdy,ROCK,2006-10-14,50.912490,44.870985,74.845,10.0,0,m,2006.90080,False
1,'N Sync,(God Must Have Spent) A Little More Time On You,POP,1998-12-05,81.288336,39.382591,62.655,0.0,12,m,1999.09920,True
2,'N Sync,Bye Bye Bye,POP,2000-01-29,71.402847,30.184848,64.441,0.0,14,m,2000.24224,True
3,'N Sync,It's Gonna Be Me,POP,2000-05-06,68.789522,53.089582,64.860,0.0,0,m,2000.51040,True
4,'N Sync,This I Promise You,POP,2000-09-30,74.214520,30.421922,65.481,0.0,14,m,2000.90784,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5011,matchbox twenty,Unwell,ROCK,2003-03-22,56.149341,35.176103,69.317,10.0,7,m,2003.36288,False
5012,twenty one pilots,Heathens,ROCK,2016-07-09,58.202752,54.398868,89.969,10.0,0,m,2016.58016,True
5013,twenty one pilots,Ride,ROCK,2016-04-02,48.387025,35.286120,89.550,10.0,1,m,2016.31200,False
5014,twenty one pilots,Stressed Out,ROCK,2015-05-16,60.782772,37.133766,88.183,10.0,1,m,2015.43712,False


In [98]:
contains_love_df[
    contains_love_df.song == 'WAP'
]

,performer,song,generic_genre,chart_debut,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,near_neighbors_count,gender,date_as_decimal,is_love_song
752,Cardi B Featuring Megan Thee Stallion,WAP,POP,2020-08-22,38.099264,42.503936,96.366,0.0,13,f,2020.67424,False


# Setup Chat GPT API calls

In [99]:
import os
import openai
# openai.organization = "personal"
openai.api_key = os.getenv("OPENAI_API_KEY")
# [model.id for model in openai.Model.list()['data']]

In [100]:
LOVE_SONG_LABELING_PROMPT = """
I will to give you a JSON list where each item is an array that contains an array where the first item is the performer and the second is the name of one of their songs from the Billboard Top 10 from 1958-2021.

Here's an example input:

```
TEST_SET_ROWS = [
    ['twenty one pilots', 'Heathens'],
    ['"Weird Al" Yankovic', 'White & Nerdy'],
    ['will.i.am & Britney Spears','Scream & Shout'],
    ["21 Savage & Metro Boomin", "Runnin"],
    ["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
    ['2Pac', "Dear Mama/Old School"],
  
    ["'N Sync", 'This I Promise You'],
    ["'N Sync & Gloria Estefan", "Music Of My Heart"],
    ["112", "Peaches & Cream"],
    ["10cc","I'm Not In Love"],
    ["50 Cent Featuring Nate Dogg", "21 Questions"],
]
```

In each array, I want you to respond by 
1. inserting "love" if it is a love song, inserting "no" if it is not, and "sex" if the song primarily revolves around human sexuality or sexual desire.
2. in the second index, add a brief justification for your classification
3. (the peformer and song name follow)

I want you to use Wikipedia's definition of a "love song": "A love song is a song about romantic love, falling in love, heartbreak after a breakup, and the feelings that these experiences bring." 

Here's an example correct response given the input above.

```
VALIDATION_SET_ROWS = [
    ['no', 'about social outcasts', 'twenty one pilots', 'Heathens'],
    ['no', 'parody about nerd culture', '"Weird Al" Yankovic', 'White & Nerdy'],
    ['no', 'generic dance tune', 'will.i.am & Britney Spears','Scream & Shout'],
    ['no', 'gangster rap', "21 Savage & Metro Boomin", "Runnin"],
    ['sex', 'about sexual prowess and uses many sexual references', "6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
    ['no', 'love for his mother, not romantic love', '2Pac', "Dear Mama/Old School"],
  
    ['love', 'promising to always love someone',"'N Sync", 'This I Promise You'],
    ['love', 'gratitude for how a lover has changed them', "'N Sync & Gloria Estefan", "Music Of My Heart"],
    ['sex', 'a song about sex & being "addicted" to a "tasty" woman', "112", "Peaches & Cream"],
    ['love', 'the title is ironic, he is actually in love with her (his wife)', "10cc","I'm Not In Love"],
    ['love', 'asking for fidelity & support no matter what', "50 Cent Featuring Nate Dogg", "21 Questions"],
]
```

It is crucial that you respond ONLY with the correct array in valid JSON format, do not include anything else in your response. In other words, your responce should be able to compiled to valid Python, do not include extraneous text outside the responce JSON array.
"""


CUT = """
Ok, start with this:

```
TEST_SET_ROWS = [
    ['100 Proof Aged in Soul', "Somebody's Been Sleeping"],
    ["50 Cent", "Disco Inferno"],
    ["24kGoldn Featuring iann dior", "Mood"],
  
    ["'N Sync Featuring Nelly", "Girlfriend"],
    ['702',"Get It Together"],
    ['10cc', "The Things We Do For Love"],
    ['Cardi B Featuring Megan Thee Stallion', 'WAP'],
]
```
"""

In [101]:
import ast
ast.literal_eval("['Michael Jackson']")
def eval_this_str(s):
    return ast.literal_eval(s)

In [102]:
unique_songs_as_numpy_array = contains_love_df[['performer', 'song']].to_numpy()
unique_songs_as_numpy_array

array([['"Weird Al" Yankovic', 'White & Nerdy'],
       ["'N Sync", '(God Must Have Spent) A Little More Time On You'],
       ["'N Sync", 'Bye Bye Bye'],
       ...,
       ['twenty one pilots', 'Ride'],
       ['twenty one pilots', 'Stressed Out'],
       ['will.i.am & Britney Spears', 'Scream & Shout']], dtype=object)

In [103]:
# TEST_SUBSET_FROM_DF = unique_songs_as_numpy_array[1:3]
# np.array2string(TEST_SUBSET_FROM_DF)
# str(list(TEST_SUBSET_FROM_DF))

TEST_SUBSET_FROM_DF = contains_love_df[['performer', 'song']][0 : 10].to_json(orient='values')
TEST_SUBSET_FROM_DF

'[["\\"Weird Al\\" Yankovic","White & Nerdy"],["\'N Sync","(God Must Have Spent) A Little More Time On You"],["\'N Sync","Bye Bye Bye"],["\'N Sync","It\'s Gonna Be Me"],["\'N Sync","This I Promise You"],["\'N Sync & Gloria Estefan","Music Of My Heart"],["\'N Sync Featuring Nelly","Girlfriend"],["\'Til Tuesday","Voices Carry"],["100 Proof Aged in Soul","Somebody\'s Been Sleeping"],["10cc","I\'m Not In Love"]]'

In [104]:
def get_love_song_labels_for_rows(row_subset_str):
    try: 
        completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "user", "content": LOVE_SONG_LABELING_PROMPT},
            {"role": "user", "content": row_subset_str},
          ]
        )
        return eval_this_str(completion.choices[0].message['content'])
    except Exception as e:
        print('ERROR FETCHING', e)
        return []

# res = get_love_song_labels_for_rows(row_subset_str=TEST_SUBSET_FROM_DF)

In [12]:
# res

[['no', 'parody about nerd culture', '"Weird Al" Yankovic', 'White & Nerdy'],
 ['love',
  "cherishing someone's presence",
  "'N Sync",
  '(God Must Have Spent) A Little More Time On You'],
 ['no', 'breakup anthem', "'N Sync", 'Bye Bye Bye'],
 ['no',
  'expresses determination and confidence',
  "'N Sync",
  "It's Gonna Be Me"],
 ['love', 'promising to always love someone', "'N Sync", 'This I Promise You'],
 ['love',
  'gratitude for how a lover has changed them',
  "'N Sync & Gloria Estefan",
  'Music Of My Heart'],
 ['no', 'flirting and attraction', "'N Sync Featuring Nelly", 'Girlfriend'],
 ['no',
  'about a toxic and abusive relationship',
  "'Til Tuesday",
  'Voices Carry'],
 ['no',
  'expressing suspicion of infidelity',
  '100 Proof Aged in Soul',
  "Somebody's Been Sleeping"],
 ['love',
  'the title is ironic, he is actually in love with her (his wife)',
  '10cc',
  "I'm Not In Love"]]

# Do full fetch, in batches

In [13]:
INCREMENT = 50

In [14]:
len(contains_love_df) / INCREMENT

100.32

In [15]:

gpt_labeled_result = []
for i in range(101):
    subset = contains_love_df[['performer', 'song']][i*INCREMENT : (i+1)*INCREMENT]

    if len(subset):
        subset_as_json_string = subset.to_json(orient='values')
        print([i*INCREMENT, (i+1)*INCREMENT])
        print(subset_as_json_string, '\n')
        fetched_and_formatted_res = get_love_song_labels_for_rows(subset_as_json_string)
        gpt_labeled_result = [*gpt_labeled_result, *fetched_and_formatted_res]

[0, 50]
[["\"Weird Al\" Yankovic","White & Nerdy"],["'N Sync","(God Must Have Spent) A Little More Time On You"],["'N Sync","Bye Bye Bye"],["'N Sync","It's Gonna Be Me"],["'N Sync","This I Promise You"],["'N Sync & Gloria Estefan","Music Of My Heart"],["'N Sync Featuring Nelly","Girlfriend"],["'Til Tuesday","Voices Carry"],["100 Proof Aged in Soul","Somebody's Been Sleeping"],["10cc","I'm Not In Love"],["10cc","The Things We Do For Love"],["112","It's Over Now"],["112","Peaches & Cream"],["1910 Fruitgum Co.","1, 2, 3, Red Light"],["1910 Fruitgum Co.","Indian Giver"],["1910 Fruitgum Co.","Simon Says"],["21 Savage & Metro Boomin","Runnin"],["21 Savage & Metro Boomin Featuring Drake","Mr. Right Now"],["24kGoldn Featuring iann dior","Mood"],["2Pac","Dear Mama\/Old School"],["2Pac Featuring K-Ci And JoJo","How Do U Want It\/California Love"],["3 Doors Down","Here Without You"],["3 Doors Down","Kryptonite"],["3 Doors Down","When I'm Gone"],["38 Special","Caught Up In You"],["3OH!3","Don't Tr

[250, 300]
[["Artists For Haiti","We Are The World 25: For Haiti"],["Ashanti","Foolish"],["Ashanti","Happy"],["Ashanti","Rain On Me"],["Ashanti","Rock Wit U (Awww Baby)"],["Ashlee Simpson","Pieces Of Me"],["Asia","Don't Cry"],["Asia","Heat Of The Moment"],["Atlanta Rhythm Section","Imaginary Lover"],["Atlanta Rhythm Section","So In To You"],["Atlantic Starr","Always"],["Atlantic Starr","Masterpiece"],["Atlantic Starr","Secret Lovers"],["Austin Roberts","Rocky"],["Ava Max","Sweet But Psycho"],["Avicii","Wake Me Up!"],["Avril Lavigne","Complicated"],["Avril Lavigne","Girlfriend"],["Avril Lavigne","I'm With You"],["Avril Lavigne","My Happy Ending"],["Avril Lavigne","Sk8er Boi"],["Az Yet","Last Night (From \"The Nutty Professor\")"],["Az Yet Featuring Peter Cetera","Hard To Say I'm Sorry"],["B*Witched","C'est La Vie"],["B-Rock & The Bizz","My Baby Daddy"],["B.J. Thomas","(Hey Won't You Play) Another Somebody Done Somebody Wrong Song"],["B.J. Thomas","Hooked On A Feeling"],["B.J. Thomas","I

[500, 550]
[["Blueface","Thotiana"],["Blues Image","Ride Captain Ride"],["Blues Magoos","(We Ain't Got) Nothin' Yet"],["Blues Traveler","Run-Around"],["Bo Bice","Inside Your Heaven"],["Bo Donaldson And The Heywoods","Billy, Don't Be A Hero"],["Bob B. Soxx And The Blue Jeans","Zip-A-Dee Doo-Dah"],["Bob Dylan","Lay Lady Lay"],["Bob Dylan","Like A Rolling Stone"],["Bob Dylan","Positively 4th Street"],["Bob Dylan","Rainy Day Women #12 & 35"],["Bob Lind","Elusive Butterfly"],["Bob Luman","Let's Think About Living"],["Bob Moore and His Orch.","Mexico"],["Bob Seger","Against The Wind"],["Bob Seger","Fire Lake"],["Bob Seger","Night Moves"],["Bob Seger","Shakedown (From \"Beverly Hills Cop II\")"],["Bob Seger","Tryin' To Live My Life Without You"],["Bob Seger & The Silver Bullet Band","Shame On The Moon"],["Bob Seger & The Silver Bullet Band","Still The Same"],["Bob Welch","Sentimental Lady"],["Bobbie Gentry","Ode To Billie Joe"],["Bobby \"Boris\" Pickett And The Crypt-Kickers","Monster Mash"],

[750, 800]
[["Cardi B","Up"],["Cardi B & Bruno Mars","Please Me"],["Cardi B Featuring Megan Thee Stallion","WAP"],["Cardi B, Bad Bunny & J Balvin","I Like It"],["Carl Anderson & Gloria Loring","Friends And Lovers"],["Carl Carlton","Everlasting Love"],["Carl Dobkins, Jr.","My Heart Is An Open Book"],["Carl Douglas","Kung Fu Fighting"],["Carla Thomas","Gee Whiz (Look At His Eyes)"],["Carly Rae Jepsen","Call Me Maybe"],["Carly Simon","Nobody Does It Better"],["Carly Simon","That's The Way I've Always Heard It Should Be"],["Carly Simon","You Belong To Me"],["Carly Simon","You're So Vain"],["Carly Simon & James Taylor","Mockingbird"],["Carole King","It's Too Late\/I Feel The Earth Move"],["Carole King","Jazzman"],["Carole King","Nightingale"],["Carole King","Sweet Seasons"],["Carolina Gaitan, Mauro Castillo, Adassa, Rhenzy Feliz, Diane Guerrero, Stephanie Beatriz & Encanto Cast","We Don't Talk About Bruno"],["Carpenters","(They Long To Be) Close To You"],["Carpenters","For All We Know"],["C

[1000, 1050]
[["Creedence Clearwater Revival","Green River"],["Creedence Clearwater Revival","Have You Ever Seen The Rain\/Hey Tonight"],["Creedence Clearwater Revival","Lookin' Out My Back Door\/Long As I Can See The Light"],["Creedence Clearwater Revival","Proud Mary"],["Creedence Clearwater Revival","Sweet Hitch-Hiker"],["Creedence Clearwater Revival","Travelin' Band\/Who'll Stop The Rain"],["Creedence Clearwater Revival","Up Around The Bend\/Run Through The Jungle"],["Crispian St. Peters","The Pied Piper"],["Crosby, Stills & Nash","Just A Song Before I Go"],["Crosby, Stills & Nash","Wasted On The Way"],["Crowded House","Don't Dream It's Over"],["Crowded House","Something So Strong"],["Crystal Gayle","Don't It Make My Brown Eyes Blue"],["Crystal Waters","Gypsy Woman (She's Homeless)"],["Culture Club","Church Of The Poison Mind"],["Culture Club","Do You Really Want To Hurt Me"],["Culture Club","I'll Tumble 4 Ya"],["Culture Club","Karma Chameleon"],["Culture Club","Miss Me Blind"],["C

...

[3900, 3950]
[["Staind","It's Been Awhile"],["Stampeders","Sweet City Woman"],["Stan Getz\/Astrud Gilberto","The Girl From Ipanema"],["Starbuck","Moonlight Feels Right"],["Starland Vocal Band","Afternoon Delight"],["Stars On 45","Medley"],["Starship","It's Not Over ('til It's Over)"],["Starship","Nothing's Gonna Stop Us Now"],["Starship","Sara"],["Starship","We Built This City"],["Stealers Wheel","Stuck In The Middle With You"],["Steam","Na Na Hey Hey Kiss Him Goodbye"],["Steely Dan","Do It Again"],["Steely Dan","Hey Nineteen"],["Steely Dan","Rikki Don't Lose That Number"],["Stephanie Mills","Never Knew Love Like This Before"],["Steppenwolf","Born To Be Wild"],["Steppenwolf","Magic Carpet Ride"],["Steppenwolf","Rock Me"],["Steve Lacy","Bad Habit"],["Steve Lawrence","Footsteps"],["Steve Lawrence","Go Away Little Girl"],["Steve Lawrence","Portrait Of My Love"],["Steve Lawrence","Pretty Blue Eyes"],["Steve Miller","Fly Like An Eagle"],["Steve Miller","Rock'n Me"],["Steve Perry","Oh, Sherr

[4150, 4200]
[["The Bachelors","Diane"],["The Bangles","Eternal Flame"],["The Bangles","Hazy Shade Of Winter"],["The Bangles","In Your Room"],["The Bangles","Manic Monday"],["The Bangles","Walk Like An Egyptian"],["The Beach Boys","Barbara Ann"],["The Beach Boys","Be True To Your School"],["The Beach Boys","California Girls"],["The Beach Boys","Dance, Dance, Dance"],["The Beach Boys","Fun, Fun, Fun"],["The Beach Boys","Good Vibrations"],["The Beach Boys","Help Me, Rhonda"],["The Beach Boys","I Get Around"],["The Beach Boys","Kokomo (From\"Cocktail\" )"],["The Beach Boys","Rock And Roll Music"],["The Beach Boys","Sloop John B"],["The Beach Boys","Surfer Girl"],["The Beach Boys","Surfin' U.S.A."],["The Beach Boys","When I Grow Up (To Be A Man)"],["The Beach Boys","Wouldn't It Be Nice"],["The Beatles","A Hard Day's Night"],["The Beatles","All You Need Is Love"],["The Beatles","Can't Buy Me Love"],["The Beatles","Come Together\/Something"],["The Beatles","Day Tripper"],["The Beatles","Do Y

[4400, 4450]
[["The Jacksons","Enjoy Yourself"],["The Jacksons","Shake Your Body (Down To The Ground)"],["The Jacksons","State of Shock"],["The Jaggerz","The Rapper"],["The Jaynetts","Sally, Go 'round The Roses"],["The Jeff Healey Band","Angel Eyes"],["The Jelly Beans","I Wanna Love Him So Bad"],["The Jets","Cross My Broken Heart (From \"Beverly Hills Cop II\")"],["The Jets","Crush On You"],["The Jets","Make It Real"],["The Jets","Rocket 2 U"],["The Jets","You Got It All"],["The Jimmy Castor Bunch","Troglodyte (Cave Man)"],["The Jive Five With Joe Rene And Orchestra","My True Story"],["The Johnny Otis Show","Willie And The Hand Jive"],["The KLF","3 A.M. Eternal"],["The Kid LAROI","Without You"],["The Kid LAROI & Justin Bieber","Stay"],["The Killers","Mr. Brightside"],["The Kingsmen","Louie Louie"],["The Kingsmen","The Jolly Green Giant"],["The Kingston Trio","Tom Dooley"],["The Kinks","All Day And All Of The Night"],["The Kinks","Come Dancing"],["The Kinks","Lola"],["The Kinks","Tired 

[4600, 4650]
[["The Spinners","I'll Be Around"],["The Spinners","The Rubberband Man"],["The Spinners","They Just Can't Stop It the (Games People Play)"],["The Spinners","Working My Way Back To You\/Forgive Me, Girl"],["The Staple Singers","I'll Take You There"],["The Staple Singers","If You're Ready (Come Go With Me)"],["The Staple Singers","Let's Do It Again"],["The Statler Brothers","Flowers On The Wall"],["The Steve Miller Band","Abracadabra"],["The Steve Miller Band","Jet Airliner"],["The Steve Miller Band","The Joker"],["The String-A-Longs","Wheels"],["The Stylistics","Break Up To Make Up"],["The Stylistics","I'm Stone In Love With You"],["The Stylistics","You Are Everything"],["The Stylistics","You Make Me Feel Brand New"],["The Stylistics Featuring Russell Thompkins,Jr.","Betcha By Golly, Wow"],["The Supremes","Baby Love"],["The Supremes","Back In My Arms Again"],["The Supremes","Come See About Me"],["The Supremes","I Hear A Symphony"],["The Supremes","Love Is Here And Now You'r

[4850, 4900]
[["Usher","U Got It Bad"],["Usher","U Remind Me"],["Usher","You Make Me Wanna..."],["Usher And Alicia Keys","My Boo"],["Usher Featuring Lil Jon & Ludacris","Yeah!"],["Usher Featuring Pitbull","DJ Got Us Fallin' In Love"],["Usher Featuring Young Jeezy","Love In This Club"],["Usher Featuring will.i.am","OMG"],["Van Halen","Jump"],["Van Halen","When It's Love"],["Van Halen","Why Can't This Be Love"],["Van McCoy And The Soul City Symphony","The Hustle"],["Van Morrison","Brown Eyed Girl"],["Van Morrison","Domino"],["Vanessa Carlton","A Thousand Miles"],["Vanessa Williams","Colors Of The Wind (From \"Pocahontas\")"],["Vanessa Williams","Dreamin'"],["Vanessa Williams","Save The Best For Last"],["Vanessa Williams\/Brian McKnight","Love Is (From \"Beverly Hills, 90210\")"],["Vangelis","Chariots Of Fire - Titles"],["Vanilla Ice","Ice Ice Baby"],["Vanilla Ice","Play That Funky Music"],["Vanity Fare","Hitchin' A Ride"],["Vertical Horizon","Everything You Want"],["Vic Dana","Red Roses 

In [16]:
# eval_this_str(resFromLastAttempt)
gpt_labeled_result

[['no', 'parody about nerd culture', '"Weird Al" Yankovic', 'White & Nerdy'],
 ['love',
  'promising to spend more time and effort on someone',
  "'N Sync",
  '(God Must Have Spent) A Little More Time On You'],
 ['no', 'breakup song, not romantic love', "'N Sync", 'Bye Bye Bye'],
 ['no',
  'about desire and anticipation, not romantic love',
  "'N Sync",
  "It's Gonna Be Me"],
 ['love', 'promising to always love someone', "'N Sync", 'This I Promise You'],
 ['love',
  'gratitude for how a lover has changed them',
  "'N Sync & Gloria Estefan",
  'Music Of My Heart'],
 ['no',
  'pop song about infatuation, not romantic love',
  "'N Sync Featuring Nelly",
  'Girlfriend'],
 ['love', 'expressing love and desire', "'Til Tuesday", 'Voices Carry'],
 ['no',
  'about suspicion and jealousy, not romantic love',
  '100 Proof Aged in Soul',
  "Somebody's Been Sleeping"],
 ['love',
  'the title is ironic, he is actually in love with her (his wife)',
  '10cc',
  "I'm Not In Love"],
 ['love',
  'express

In [17]:
gpt_labeled_result[1]

['love',
 'promising to spend more time and effort on someone',
 "'N Sync",
 '(God Must Have Spent) A Little More Time On You']

In [20]:
first_label_pass_df = pd.DataFrame(gpt_labeled_result, columns=['is_love_or_sex_song', 'rational', 'performer', 'song'])

In [22]:
# first_label_pass_df.to_csv('./data/15-TEMP-OUTPUT-love-song-label-first-pass.csv', index=False)

### Re-fetch any that we're still missing

In [39]:
# take the non-overlap with the new & original data
merged_df = first_label_pass_df.merge(contains_love_df, on=['performer', 'song'], how='right', indicator=True)
only_in_right_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns='_merge')
only_in_right_df
# re-fetch just those

,is_love_or_sex_song,rational,performer,song,generic_genre,chart_debut,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,near_neighbors_count,gender,date_as_decimal,is_love_song
129,NaN,NaN,Alanis Morissette,You Learn/You Oughta Know,POP,1996-07-27,53.735808,64.582755,58.991,0.0,0,f,1996.75424,True
194,NaN,NaN,Ann Wilson & Robin Zander,"Surrender To Me (From ""Tequila Sunrise"")",UNCATEGORIZED,1988-12-24,36.765062,66.276575,47.205,40.0,4,x,1989.21120,True
305,NaN,NaN,Bachman-Turner Overdrive,You Ain't Seen Nothing Yet/Free Wheelin',ROCK,1974-09-21,36.856044,57.376051,25.062,10.0,1,m,1975.03968,True
338,NaN,NaN,Barbra Streisand,The Main Event/Fight,ELECTRO,1979-06-16,52.561771,51.365098,32.422,50.0,0,f,1979.75008,True
345,NaN,NaN,Barbra Streisand/Donna Summer,No More Tears (Enough Is Enough),ELECTRO,1979-10-20,74.002973,33.840032,32.950,50.0,0,f,1980.08800,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4362,NaN,NaN,The Guess Who,American Woman/No Sugar Tonight,ROCK,1970-03-21,58.205526,52.479981,18.075,10.0,1,m,1970.56800,False
4431,NaN,NaN,The Lettermen,Goin' Out Of My Head/Can't Take My Eyes Off You,POP,1967-12-09,55.246890,38.018444,14.534,0.0,0,m,1968.30176,True
4492,NaN,NaN,The O'Jays,Use Ta Be My Girl,JAZZ_INFLUENCED,1978-04-29,77.458051,33.004253,30.668,30.0,13,m,1978.62752,True
4868,NaN,NaN,Vanessa Williams/Brian McKnight,"Love Is (From ""Beverly Hills, 90210"")",JAZZ_INFLUENCED,1993-01-23,42.323543,42.352681,53.540,30.0,0,x,1993.26560,True


In [43]:
gpt_labeled_result2 = []
for i in range(10):
    subset = only_in_right_df[['performer', 'song']][i*INCREMENT : (i+1)*INCREMENT]

    if len(subset):
        subset_as_json_string = subset.to_json(orient='values')
        print([i*INCREMENT, (i+1)*INCREMENT])
        print(subset_as_json_string, '\n')
        fetched_and_formatted_res = get_love_song_labels_for_rows(subset_as_json_string)
        gpt_labeled_result2 = [*gpt_labeled_result2, *fetched_and_formatted_res]

gpt_labeled_result2


[0, 50]
[["Alanis Morissette","You Learn\/You Oughta Know"],["Ann Wilson & Robin Zander","Surrender To Me (From \"Tequila Sunrise\")"],["Bachman-Turner Overdrive","You Ain't Seen Nothing Yet\/Free Wheelin'"],["Barbra Streisand","The Main Event\/Fight"],["Barbra Streisand\/Donna Summer","No More Tears (Enough Is Enough)"],["Berlin","Take My Breath Away (Love Theme From \"Top Gun\")"],["Bette Midler","Wind Beneath My Wings (From \"Beaches\")"],["Big Mountain","Baby I Love Your Way (From \"Reality Bites\")"],["Billy Idol","Cradle Of Love (From \"Ford Fairlane\")"],["Billy Ocean","When The Going Gets Tough, The Tough Get Going"],["Brook Benton","Rainy Night In Georgia\/Rubberneckin'"],["Bryan Adams\/Rod Stewart\/Sting","All For Love"],["Busta Rhymes","Turn It Up [Remix]\/Fire It Up"],["Busta Rhymes","Woo-Hah!! Got You All In Check\/Everything Remains Raw"],["C+C Music Factory\/F. Williams","Here We Go"],["Chicago","Beginnings\/Colour My World"],["Cyndi Lauper","The Goonies `R' Good Enough"

[['no',
  'a song about personal growth and self-reflection',
  'Alanis Morissette',
  'You Learn\\/You Oughta Know'],
 ['no',
  'a duet about the struggles of love',
  'Ann Wilson & Robin Zander',
  'Surrender To Me (From "Tequila Sunrise")'],
 ['no',
  'a song about believing in yourself and overcoming challenges',
  'Bachman-Turner Overdrive',
  "You Ain't Seen Nothing Yet\\/Free Wheelin'"],
 ['no',
  'a song about a boxing match',
  'Barbra Streisand',
  'The Main Event\\/Fight'],
 ['no',
  'a duet about female empowerment',
  'Barbra Streisand\\/Donna Summer',
  'No More Tears (Enough Is Enough)'],
 ['love',
  'a romantic song from a movie soundtrack',
  'Berlin',
  'Take My Breath Away (Love Theme From "Top Gun")'],
 ['love',
  'a song about gratitude for a supportive friend',
  'Bette Midler',
  'Wind Beneath My Wings (From "Beaches")'],
 ['love',
  'a cover of a classic love song',
  'Big Mountain',
  'Baby I Love Your Way (From "Reality Bites")'],
 ['no',
  'a song about desir

In [44]:
second_label_pass_df = pd.DataFrame(gpt_labeled_result2, columns=['is_love_or_sex_song', 'rational', 'performer', 'song'])
len(second_label_pass_df)


187

In [73]:
combined_labeled_df = pd.concat([first_label_pass_df, second_label_pass_df])
combined_labeled_df[['performer', 'song']] = combined_labeled_df[['performer', 'song']].replace(
    {
        '\\\\/': '/', 
#         '\'': '\"'
    }, 
    regex=True
)


combined_labeled_df


,is_love_or_sex_song,rational,performer,song
0,no,parody about nerd culture,"""Weird Al"" Yankovic",White & Nerdy
1,love,promising to spend more time and effort on som...,'N Sync,(God Must Have Spent) A Little More Time On You
2,no,"breakup song, not romantic love",'N Sync,Bye Bye Bye
3,no,"about desire and anticipation, not romantic love",'N Sync,It's Gonna Be Me
4,love,promising to always love someone,'N Sync,This I Promise You
...,...,...,...,...
182,no,a song about freedom and expressing oneself,The Guess Who,American Woman/No Sugar Tonight
183,love,a song about being infatuated and losing control,The Lettermen,Goin' Out Of My Head/Can't Take My Eyes Off You
184,no,a song about missing a former lover,The O'Jays,Use Ta Be My Girl
185,love,a song about the power of love,Vanessa Williams/Brian McKnight,"Love Is (From ""Beverly Hills, 90210"")"


In [74]:
contains_love_df[
    contains_love_df.song.astype(str).str.contains('Take My Breath Away')
]

,performer,song,generic_genre,chart_debut,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,near_neighbors_count,gender,date_as_decimal,is_love_song
400,Berlin,"Take My Breath Away (Love Theme From ""Top Gun"")",UNCATEGORIZED,1986-06-21,54.92900,53.854872,43.307,40.0,0,x,1986.71648,True
3524,Rex Smith,You Take My Breath Away,ROCK,1979-04-21,50.80935,29.978888,32.174,10.0,6,m,1979.59136,False


In [85]:
# combined_labeled_df[
#     combined_labeled_df.performer.astype(str).str.contains('Florida Georgia')
# ]

with_gpt_labels_added_df = contains_love_df.merge(
    on=['performer', 'song'],
    right=combined_labeled_df,
    how='right',
    indicator=True
).drop_duplicates(subset=['performer', 'song'])

with_gpt_labels_added_df = with_gpt_labels_added_df[
    with_gpt_labels_added_df._merge == 'both'
].dropna().drop(columns='_merge').rename(columns={
    'is_love_or_sex_song': 'gpt3_song_is_love_or_sex_or_no', 
    'rational': 'gpt3_classification_rational',
    'is_love_song': 'contains_word_love'
})

In [87]:
# save data to disk, reload it (for future cold runs of hte noteboook)
with_gpt_labels_added_df.gpt3_song_is_love_or_sex_or_no.value_counts()

no      3669
love    1261
sex       84
Name: gpt3_song_is_love_or_sex_or_no, dtype: int64

In [93]:
with_gpt_labels_added_df[
    ~with_gpt_labels_added_df.chart_debut.str.startswith('2022')
].to_csv('./data/14-OUTPUT-gpt3-love-song-labels-through-2021.csv', index=False)

## Testing with Lady Gaga, who got weird results

In [110]:
gaga_songs_df = contains_love_df[
    contains_love_df.performer.str.contains('Lady Gaga')
][['performer', 'song']]

In [114]:
fetched_and_formatted_res = get_love_song_labels_for_rows(
    gaga_songs_df.to_json(orient='values')
)

pd.DataFrame(fetched_and_formatted_res, columns=['is_love_or_sex_song', 'rational', 'performer', 'song'])


,is_love_or_sex_song,rational,performer,song
0,love,a song about a complicated and passionate love...,Lady Gaga,Alejandro
1,no,a song about the desire for fame and applause,Lady Gaga,Applause
2,love,a song about an intense and toxic romantic rel...,Lady Gaga,Bad Romance
3,love,a song about self-acceptance and embracing ind...,Lady Gaga,Born This Way
4,no,a song about addiction and struggling with sub...,Lady Gaga,Dope
5,no,a song about being attracted to a bad boy,Lady Gaga,Judas
6,no,a song about sexual attraction and playing lov...,Lady Gaga,LoveGame
7,love,a song about finding strength and hope in a di...,Lady Gaga,Million Reasons
8,no,"a song about fame, obsession, and the paparazzi",Lady Gaga,Paparazzi
9,no,a song about sexual attraction and wearing a p...,Lady Gaga,Poker Face


,is_love_or_sex_song,rational,performer,song
0,love,a song about a passionate and tumultuous relat...,Lady Gaga,Alejandro
1,no,a song about fame and the performance industry,Lady Gaga,Applause
2,love,a song about a toxic and obsessive love affair,Lady Gaga,Bad Romance
3,love,self-empowerment anthem about embracing one's ...,Lady Gaga,Born This Way
4,no,a song about struggling with addiction and sel...,Lady Gaga,Dope
5,no,a song about being attracted to bad boys,Lady Gaga,Judas
6,no,a song about sexual attraction and flirtation,Lady Gaga,LoveGame
7,love,a song about finding strength and love in diff...,Lady Gaga,Million Reasons
8,no,a song about the downsides of fame and invasio...,Lady Gaga,Paparazzi
9,no,a song about seduction and playing with emotions,Lady Gaga,Poker Face


# Merging this into a format we can export to deepscatter:

In [115]:
df = pd.read_csv('./data/14-OUTPUT-gpt3-love-song-labels-through-2021.csv')

In [124]:
gender_df.merge(
    df[['performer', 'song', 'gpt3_song_is_love_or_sex_or_no']], 
    on=['performer', 'song'],
    how='left'
).drop(
    columns=['date_as_decimal', 'contains_love']
).to_csv('./data/14-OUTPUT-viz-ready-data-with-gpt3-love-song-labels.csv')


In [117]:
df[['performer', 'song', 'generic_genre', 'x', 'y', 'chart_debut_date_normalized_0_to_1', 'genre_position_band', 'gpt3_song_is_love_or_sex_or_no']]

,performer,song,generic_genre,x,y,chart_debut_date_normalized_0_to_1,genre_position_band,gpt3_song_is_love_or_sex_or_no
0,"""Weird Al"" Yankovic",White & Nerdy,ROCK,50.912490,44.870985,74.845,10.0,no
1,'N Sync,(God Must Have Spent) A Little More Time On You,POP,81.288336,39.382591,62.655,0.0,love
2,'N Sync,Bye Bye Bye,POP,71.402847,30.184848,64.441,0.0,no
3,'N Sync,It's Gonna Be Me,POP,68.789522,53.089582,64.860,0.0,no
4,'N Sync,This I Promise You,POP,74.214520,30.421922,65.481,0.0,love
...,...,...,...,...,...,...,...,...
4940,Patty Smyth With Don Henley,Sometimes Love Just Ain't Enough,UNCATEGORIZED,58.541992,56.160739,52.857,40.0,love
4941,Post Malone & Swae Lee,Sunflower (Spider-Man: Into The Spider-Verse),HIP_HOP,36.598279,48.039498,93.571,20.0,no
4942,Prince,I Could Never Take The Place Of Your Man,LATIN,56.627054,45.082954,45.481,80.0,love
4943,Queen,Another One Bites The Dust,ROCK,58.892364,40.185480,34.239,10.0,no
